In [1]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('./data/olive0_classify.csv',encoding='utf-8')
df

,brand,product,text,cate,animal
0,랩시리즈,랩시리즈 올인원 트리트먼트 50ml 1+1 기획세트,"사이클로덱스트린,포타슘스테아레이트,포도씨추출물,스쿠알란,옥틸도데실네오펜타노에이트,에...",스킨케어,1
1,유세린,유세린 하이알루론 아이크림X나이트크림 기획(아이크림 15ml+나이트크림 50ml+컨...,"에칠헥실리세린,부틸렌라이콜디카프릴레이트,이소프로필팔미테이트,하이드로제네이티드코코리세...",스킨케어,1
2,아이디얼포맨,아이디얼 포맨 퍼펙트스킨케어 2종세트(미니어처 3종 증정),"페닐알라닌,무화과추출물,홀리바질잎추출물,나이아신아마이드,카프릴릭,다이소듐이디티에이,...",스킨케어,1
3,다슈,다슈 맨즈 아쿠아 토너/로션 153ml 2종 세트(+토너&로션&클렌징 30ml증정),"개청각추출물,연꽃씨추출물,바실러스,마트리카리아꽃추출물,띠뿌리발효추출물,페닐알라닌,민...",스킨케어,1
4,달바,달바 화이트 트러플 퍼스트 아로마틱 토너 155ml,"네오펜틸글라이콜다이헵타노에이트,하이드록시에틸우레아,돌콩오일,브로멜라인,소듐클로라이드...",스킨케어,1
...,...,...,...,...,...
3748,데싱디바,데싱디바 매직프레스 베이스 쉴드,"토코페롤,스위트아몬드오일,아이소프로필알코올,해바라기씨오일,포스포릭애씨드,실리카,트라...",네일,0
3749,엣지유,엣지유 샵케어 탑젤 글로시&매트,"유칼립투스잎추출물,하이드록시사이클로헥실페닐케톤,셀룰로오스아세테이트부티레이트,헤마,매...",네일,0
3750,데싱디바,데싱디바 리치 세럼,"벤질살리실레이트,포도씨오일,미네랄오일,스타이렌코폴리머,부틸렌,에틸렌,부틸페닐메틸프로...",네일,0
3751,위드샨,위드샨 에코 네일 리무버 200ml,"제라니올,리날룰,프로필렌글라이콜,정제수,라벤더오일,토코페릴아세테이트,오레가노오일,로...",네일,0


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

df['text']=df['text'].apply(lambda x: x.replace(',',' '))
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))
text_mat=count_vect.fit_transform(df['text'])
print(text_mat.shape)

(3753, 50088)


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

text_sim=cosine_similarity(text_mat,text_mat)
print(text_sim.shape)
print(text_sim[:1])

(3753, 3753)
[[1.         0.06436227 0.0599272  ... 0.04567925 0.02283962 0.        ]]


In [5]:
text_sim_sortred_ind=text_sim.argsort()[:,::-1]
print(text_sim_sortred_ind[:1])

[[   0  151  643 ... 1901 1904 1876]]


In [6]:
def find_sim_text(df,sorted_ind,product_name,top_n=10,vegan=0,cate=0):

    title_product=df[df['product']==product_name]
    title_index=title_product.index.values

    similar_indexes=sorted_ind[title_index]
    similar_indexes=similar_indexes[similar_indexes!=title_index]

    similar_indexes=similar_indexes.reshape(-1)
    indexes=similar_indexes

    # 비건 옵션
    if vegan==1:
        vegan_indexes=df.loc[df['animal']==0].index.values

    # 카테고리 옵션
    if cate==1:
        category=df.loc[df['product']==product_name,'cate'].values
        cate_indexes=df.loc[df['cate']==category[0]].index.values
        
    # 옵션 적용
    option_lst=[]
    if vegan==1 and cate==1: # 비건, 카테고리
        for idx in similar_indexes:
            if idx in vegan_indexes and idx in cate_indexes:
                option_lst.append(idx)
        indexes=option_lst
    
    elif vegan==1 and cate==0: # 비건
        for idx in similar_indexes:
            if idx in vegan_indexes:
                option_lst.append(idx)
        indexes=option_lst

    elif vegan==0 and cate==1: # 카테고리
        for idx in similar_indexes:
            if idx in cate_indexes:
                option_lst.append(idx)
        indexes=option_lst
        
    return df.iloc[indexes][:top_n]

In [7]:
df.iloc[[1423]]

,brand,product,text,cate,animal
1423,라운드어라운드,라운드어라운드 그린티 수분 클렌징 오일 300mL,호호바씨오일 오렌지껍질오일 카프릴릭 올리브오일 리모넨 리날룰 에틸헥실글리세린 시트로...,스킨케어,0


In [10]:
find_sim_text(df,text_sim_sortred_ind,df.loc[1423,'product'],10,1,1)

,brand,product,text,cate,animal
1412,마녀공장,마녀공장 퓨어 클렌징 오일 200ml,호호바씨오일 돌콩오일 아이소아밀라우레이트 라벤더오일 카프릴릭 올리브오일 쌀발효여과물...,스킨케어,0
1420,스킨1004,스킨1004 마다가스카르 센텔라 라이트 클렌징 오일 200ml,올리브오일 호호바씨오일 리모넨 해바라기씨오일 에틸헥실글리세린 센티드제라늄꽃오일 리날...,스킨케어,0
1395,에이트루,에이트루 퓨어 밸런싱 클렌징 오일 150ml 기획,올리브오일 호호바씨오일 트로폴론 리모넨 에틸헥실글리세린 리날룰 포도씨오일 베르가모트...,스킨케어,0
1391,에이트루,에이트루 퓨어 밸런싱 클렌징 오일 300ml,올리브오일 호호바씨오일 트로폴론 리모넨 에틸헥실글리세린 리날룰 포도씨오일 베르가모트...,스킨케어,0
1353,더파이에센셜,더파이 선플라워 클렌징 오일 200ml,아이소프로필미리스테이트 로즈마리잎오일 카프릴릴글라이콜 스쿠알란 카프릭트라이글리세라이...,스킨케어,0
937,이즈앤트리,이즈앤트리 히아루론산 워터 미스트,베타글루칸 솔비탄세스퀴올리에이트 안하이드로자일리톨 카프릴릭 리모넨 리날룰 펜틸렌글라...,스킨케어,0
1380,아비브,아비브 포어 클렌징 오일 어성초 오일 워시 200ml,폴리아이소부텐 호호바씨오일 라벤더꽃추출물 아이소프로필팔미테이트 클레리오일 카프릴릭 ...,스킨케어,0
1348,마녀공장,마녀공장 퓨어 클렌징 오일 200ml+퓨어폼 20ml,호호바씨오일 돌콩오일 아이소아밀라우레이트 라벤더오일 카프릴릭 올리브오일 쌀발효여과물...,스킨케어,0
1386,라운드랩,라운드랩 약콩 클렌징오일 200ml (클렌저 20ml 증정),시트릭애씨드 소듐이세티오네이트 스쿠알란 트라이소듐에틸렌다이아민다이석시네이트 쌀가루 ...,스킨케어,0
921,브링그린,브링그린 당근 비타 잡티 세럼 45ml,홀리바질잎추출물 알부틴 잔탄검 베티버뿌리오일 나이아신아마이드 오렌지껍질오일 글리세린...,스킨케어,0
